In [1]:
import gym
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import math

In [2]:
## preprocesser une image du jeu (scale down, monochrome)
def prepro(I):
    """ preprocessing 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0,len(r))):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [15]:
class Rl(nn.Module):
        def __init__(self,input_size,h_size, output_size):
            super(Rl,self).__init__()
            self.linear1 = nn.Linear(input_size,h_size)
            self.linear2 = nn.Linear(h_size,output_size)
            self.sigmoid = nn.Sigmoid()
            
        def forward(self,x):
            h = F.relu(self.linear1(x))
            logp = self.sigmoid(self.linear2(h))
            return logp

In [31]:
#TODO : bug quand reward = 0 et du training voir karpathy : http://karpathy.github.io/2016/05/31/rl/
env = gym.make('Pong-v0') #charge l'environement
env.reset()  # reinitialiser le jeu

env.action_space # liste des actions possibles
env.unwrapped.get_action_meanings() # signification des actions
observation, reward, done, info = env.step(1) # joue l'action 1
# print(observation,done,info)
l = 0
r = 0
r_prec = 0
D = 80*80
R_model = Rl(D,200,2)
gamma = 0.99
l = 0
render = False
optimizer = optim.SGD(R_model.parameters(),lr=0.001)

for num_episode,episode in enumerate(range(5)):
    log_P_action = []
    rewards = []
    reward_sum = 0
    prec = None
    for t in range(1000):
        if render : env.render() # afficher l'etat du jeu
        
        #prepocessing et copie de l'etat precdu jeu
        current = prepro(observation)
        x =  current - prec if prec is not None else np.zeros(D)
        prec = current
        
        x = Variable(torch.FloatTensor(x))
        #calcul des probas d'action
        logP = R_model(x)
        #tirage d'une action
        action = torch.multinomial(logP.exp(),1)
        log_P_action.append(logP[action])
        
        #0  aller en bas 
        g_action = 2 if action.data[0] == 0 else 3 
#         print(action)
        observation, reward, done, info = env.step(g_action)
        
        rewards.append(reward)
        reward_sum += reward
        
    discount_reward = discount_rewards(rewards)
    discount_reward = ( discount_reward - np.mean(discount_reward)) / np.std(discount_reward)

    #calcul du loss
    loss = []
    for i in range(len(log_P_action)):
        loss.append(-log_P_action[i] * discount_reward[i])
    optimizer.zero_grad()
    print(type(loss[1]))
    print(loss[1].grad_fn)
    print(grez)
#     print(type(loss[1].data[0]))
    if math.isnan(loss[1].data):
        loss = torch.cat(loss).sum()
    else:
        loss = torch.cat(loss).sum()
    print(loss)
    loss.backward()
    optimizer.step()
    
print(l)
#en entree du reseau du neurone 


<class 'torch.autograd.variable.Variable'>


NameError: name 'grez' is not defined

In [6]:
env.action_space # liste des actions possibles
env.unwrapped.get_action_meanings() # signification des actions

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']